In [1]:
import pandas as pd
import numpy as np
from collections import Counter

In [2]:
train = [[_.split(',') for _ in item.rstrip('\n').split(';')] for item in open('coursera_sessions_train.txt').readlines()]
test = [[_.split(',') for _ in item.rstrip('\n').split(';')]  for item in open('coursera_sessions_test.txt').readlines()]

In [37]:
def write_answer(filename, result):
    f = open(filename, 'w')
    f.write(' '.join(str(round(_, 2)) for _ in result))
    f.close()

In [3]:
def get_counters_for_sample(sample):
    see = []
    buy = []
    
    for item in sample:
        see += item[0]
        buy += item[1]
    
    return Counter(see), Counter(buy)

In [13]:
def first_index(lines):
    session = dict()
    for index, item in enumerate(lines):
        if item not in session:
            session[item] = index
    return session

In [33]:
def sorted_see(see, count_see):
    return list(pd.DataFrame(
        [[key, value, count_see.get(key, 0)] for key, value in first_index(see).iteritems()],
        columns=['item', 'index', 'count']
    ).sort_values(['count', 'index'], ascending=[False, True]).item.values)

In [18]:
def intersection(left, right):
    return len(set(left).intersection(set(right)))

In [68]:
def calculate(sample, count_see):
    precision_1 = []
    recall_1 = []
    precision_5 = []
    recall_5 = []
    for see, buy in sample:
        if buy == ['']:
            continue
        

        see = sorted_see(see, count_see)
        precision_1.append(intersection([see[0]], buy))
        recall_1.append(float(intersection([see[0]], buy))/len(buy))
        precision_5.append(float(intersection(see[:5], buy))/5)
        recall_5.append(float(intersection(see[:5], buy))/len(buy))
    return map(np.average, (recall_1, precision_1, recall_5, precision_5))

In [7]:
count_see, count_buy = get_counters_for_sample(train)

In [69]:
write_answer('answer1', calculate(train, count_see))
write_answer('answer2', calculate(test, count_see))
write_answer('answer3', calculate(train, count_buy))
write_answer('answer4', calculate(test, count_buy))